In [42]:
from nba_api.stats.endpoints import playergamelog, boxscoreadvancedv2
import pandas as pd
import numpy as np
import time
import random
import sys

In [43]:
class PlayerProfile:
    def __init__(self, name, player_id, seasons):
        self.name = name
        self.id = player_id
        self.seasons = seasons
        self.gamelog = self.collect_stats(player_id, seasons)
        self.advanced_stats = self

    def __str__(self):
        return self.name  # This controls what gets printed when using print(object)

    def __repr__(self):
        return f"PlayerProfile(name={self.name!r})"
    
    def collect_stats(self, player_id, seasons):
        gamelog = self.collect_gamelog(player_id, seasons)
        gamelog = self.collect_adv_stats(gamelog, player_id)

        return gamelog

    def collect_gamelog(self, player_id, seasons):
        gamelog = pd.DataFrame()
        for season in seasons:
            data = playergamelog.PlayerGameLog(player_id=player_id, season=season)
            data = data.get_dict()
            if gamelog.empty:
                cols = data['resultSets'][0]['headers']
                gamelog = pd.DataFrame(columns=cols)
            observations = data['resultSets'][0]['rowSet']
            gamelog = pd.concat([gamelog, pd.DataFrame(observations, columns=gamelog.columns)], ignore_index=True)

        return gamelog
    
    def collect_adv_stats(self, gamelog, player_id):
        '''manage advanced stats'''
        categories = ['E_USG_PCT', 'E_OFF_RATING', 'E_DEF_RATING', 'PACE_PER40', 'TS_PCT', 'AST_PCT']

        # Ensure gamelog has the columns for advanced stats
        for col in categories:
            gamelog[col] = None  # Initialize empty columns
        
        # Iterate over each game entry
        for idx, row in gamelog.iterrows():
            game_id = row['Game_ID']
            
            try:
                # Print dynamic status update (overwriting the previous game_id)
                sys.stdout.write(f"\rProcessing game: {game_id}... ")
                sys.stdout.flush()

                # Introduce a random delay before making the API request
                sleep_time = random.uniform(1, 5)  # Sleep between 1 to 5 seconds
                time.sleep(sleep_time)
                stats = self.adv_stats_by_game(game_id, player_id)  # Retrieve stats
                for col in categories:
                    gamelog.at[idx, col] = stats[col]  # Update DataFrame
            except AssertionError as e:
                print(f"Skipping game {game_id} due to error: {e}")
            except Exception as e:
                print(f"Unexpected error retrieving stats for game {game_id}: {e}")

        return gamelog  # Return the updated DataFrame
        


    def adv_stats_by_game(self, game_id, player_id):
        categories = ['E_USG_PCT', 'E_OFF_RATING', 'E_DEF_RATING', 'PACE_PER40', 'TS_PCT', 'AST_PCT']
        print(game_id)
        print(type(game_id))
        result = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=str(game_id))
        result = result.get_dict()
        data = pd.DataFrame(
            data=result['resultSets'][0]['rowSet'],
            columns=result['resultSets'][0]['headers'])
        pl = data[data['PLAYER_ID'] == int(player_id)]
        assert len(pl) == 1, "Too many rows returned for game/player advanced stats pull."
        return pl[categories].iloc[0].to_dict()






In [44]:
SEASONS = [
    '2024-25',
    '2023-24',
    '2022-23',
    '2021-22',
    '2020-21'
]
PLAYER_ID = '1627759'
NAME = 'Jalen Brown'
player = PlayerProfile(
    name=NAME,
    player_id=PLAYER_ID,
    seasons=SEASONS
)
print(player)

/var/folders/fm/7zh52wdj2mx7v17vqtcy7ggc0000gn/T/ipykernel_95233/2772840929.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gamelog = pd.concat([gamelog, pd.DataFrame(observations, columns=gamelog.columns)], ignore_index=True)


Processing game: 0022400891... 0022400891
<class 'str'>
Processing game: 0022400866... 0022400866
<class 'str'>
Processing game: 0022400852... 0022400852
<class 'str'>
Processing game: 0022400829... 0022400829
<class 'str'>
Processing game: 0022400811... 0022400811
<class 'str'>
Processing game: 0022400789... 0022400789
<class 'str'>
Processing game: 0022400748... 0022400748
<class 'str'>
Processing game: 0022400728... 0022400728
<class 'str'>
Processing game: 0022400710... 0022400710
<class 'str'>
Processing game: 0022400698... 0022400698
<class 'str'>
Processing game: 0022400683... 0022400683
<class 'str'>
Processing game: 0022400666... 0022400666
<class 'str'>
Processing game: 0022400650... 0022400650
<class 'str'>
Processing game: 0022400635... 0022400635
<class 'str'>
Processing game: 0022400629... 0022400629
<class 'str'>
Processing game: 0022400620... 0022400620
<class 'str'>
Processing game: 0022400604... 0022400604
<class 'str'>
Processing game: 0022400587... 0022400587
<class

In [49]:
player.gamelog

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,E_USG_PCT,E_OFF_RATING,E_DEF_RATING,PACE_PER40,TS_PCT,AST_PCT
0,22024,1627759,0022400891,"MAR 05, 2025",BOS vs. POR,W,40,7,23,0.304,...,2,18,15,1,0.31,139.4,119.8,79.08,0.351,0.25
1,22024,1627759,0022400866,"MAR 02, 2025",BOS vs. DEN,W,37,6,15,0.400,...,2,22,10,1,0.307,121.8,103.9,79.23,0.58,0.348
2,22024,1627759,0022400852,"FEB 28, 2025",BOS vs. CLE,L,34,13,24,0.542,...,4,37,10,1,0.399,128.9,115.4,78.43,0.651,0.111
3,22024,1627759,0022400829,"FEB 25, 2025",BOS @ TOR,W,33,10,18,0.556,...,1,24,5,1,0.287,110.4,110.7,80.2,0.607,0.19
4,22024,1627759,0022400811,"FEB 23, 2025",BOS vs. NYK,W,37,10,21,0.476,...,2,24,14,1,0.293,121.4,100.5,81.38,0.548,0.083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,22020,1627759,0022000056,"DEC 30, 2020",BOS vs. MEM,W,29,15,21,0.714,...,1,42,22,1,0.378,122.7,81.8,87.74,0.888,0.267
309,22020,1627759,0022000048,"DEC 29, 2020",BOS @ IND,W,34,7,13,0.538,...,2,20,-1,1,0.231,114.0,109.3,86.13,0.575,0.105
310,22020,1627759,0022000037,"DEC 27, 2020",BOS @ IND,L,34,8,13,0.615,...,4,18,1,1,0.206,99.1,98.3,86.88,0.648,0.235
311,22020,1627759,0022000007,"DEC 25, 2020",BOS vs. BKN,L,31,11,25,0.440,...,5,27,-25,1,0.415,88.1,132.3,81.92,0.466,0.273
